<a href="https://colab.research.google.com/github/venkatasl/AIML_TRAINING_VENKAT/blob/main/Day_04_Part_2_CNN_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Convolutional Neural Networks

## 1. Convolutions

It turns out that there is a very efficient way of pulling this off, and it makes advantage of the structure of the information encoded within an image---it is assumed that pixels that are spatially *closer* together will "cooperate" on forming a particular feature of interest much more than ones on opposite corners of the image. Also, if a particular (smaller) feature is found to be of great importance when defining an image's label, it will be equally important if this feature was found anywhere within the image, regardless of location.

Enter the **convolution** operator. Given a two-dimensional image, $\bf I$, and a small matrix, $\bf K$ of size $h \times w$, (known as a *convolution kernel*), which we assume encodes a way of extracting an interesting image feature, we compute the convolved image, ${\bf I} * {\bf K}$, by overlaying the kernel on top of the image in all possible ways, and recording the sum of elementwise products between the image and the kernel:

$$({\bf I} * {\bf K})_{xy} = \sum_{i=1}^h \sum_{j=1}^w {{\bf K}_{ij} \cdot {\bf I}_{x + i - 1, y + j - 1}}$$

(in fact, the exact definition would require us to flip the kernel matrix first, but for the purposes of machine learning it is irrelevant whether this is done)

The images below show a diagrammatical overview of the above formula and the result of applying convolution (with two separate kernels) over an image, to act as an edge detector:

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/convolve.png)
![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/lena.jpg)

## 2. Convolutional layers and Pooling layers

The convolution operator forms the fundamental basis of the **convolutional** layer of a CNN. The layer is completely specified by a certain number of kernels, $\bf \vec{K}$ (along with additive biases, $\vec{b}$, per each kernel), and it operates by computing the convolution of the output images of a previous layer with each of those kernels, afterwards adding the biases (one per each output image). Finally, an activation function, $\sigma$, may be applied to all of the pixels of the output images. Typically, the input to a convolutional layer will have $d$ *channels* (e.g. red/green/blue in the input layer), in which case the kernels are extended to have this number of channels as well, making the final formula of a single output image channel of a convolutional layer (for a kernel ${\bf K}$ and bias $b$) as follows:

$$\mathrm{conv}({\bf I}, {\bf K})_{xy} = \sigma\left(b + \sum_{i=1}^h \sum_{j=1}^w \sum_{k=1}^d {{\bf K}_{ijk} \cdot {\bf I}_{x + i - 1, y + j - 1, k}}\right)$$

Note that, since all we're doing here is addition and scaling of the input pixels, the kernels may be learned from a given training dataset via *gradient descent*, exactly as the weights of an MLP. In fact, an MLP is perfectly capable of replicating a convolutional layer, but it would require a lot more training time (and data) to learn to approximate that mode of operation.

Finally, let's just note that a convolutional operator is in no way restricted to two-dimensionally structured data: in fact, most machine learning frameworks ([Keras included](https://keras.io/layers/convolutional/)) will provide you with out-of-the-box layers for 1D and 3D convolutions as well!

It is important to note that, while a convolutional layer significantly decreases the number of *parameters* compared to a fully connected (FC) layer, it introduces more **hyperparameters**---parameters whose values need to be chosen *before* training starts.

Namely, the hyperparameters to choose within a single convolutional layer are:
- *depth*: how many different kernels (and biases) will be convolved with the output of the previous layer;
- *height* and *width* of each kernel;
- *stride*: by how much we shift the kernel in each step to compute the next pixel in the result. This specifies the overlap between individual output pixels, and typically it is set to $1$, corresponding to the formula given before. Note that larger strides result in smaller output sizes.
- *padding*: note that convolution by any kernel larger than $1\times 1$ will *decrease* the output image size---it is often desirable to keep sizes the same, in which case the image is sufficiently padded with zeroes at the edges. This is often called *"same"* padding, as opposed to *"valid"* (no) padding. It is possible to add arbitrary levels of padding, but typically the padding of choice will be either same or valid.

As already hinted, convolutions are not typically meant to be the sole operation in a CNN (although there have been promising recent developments on [all-convolutional networks](https://arxiv.org/pdf/1412.6806v3.pdf)); but rather to extract useful features of an image prior to downsampling it sufficiently to be manageable by an MLP.

A very popular approach to downsampling is a *pooling* layer, which consumes small and (usually) disjoint chunks of the image (typically $2\times 2$) and aggregates them into a single value. There are several possible schemes for the aggregation---the most popular being **max-pooling**, where the maximum pixel value within each chunk is taken. A diagrammatical illustration of $2\times 2$ max-pooling is given below.

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/pool.png)

## 3. Convolutional Neural Network (CNN) architecture

Now that we got all the building blocks, let's see what a typical convolutional neural network might look like!

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/cnn.png)

A typical CNN architecture for a $k$-class image classification can be split into two distinct parts---a chain of repeating $\mathrm{Conv}\rightarrow\mathrm{Pool}$ layers (sometimes with more than one convolutional layer at once), followed by a few fully connected layers (taking each pixel of the computed images as an independent input), culminating in a $k$-way softmax layer, to which a cross-entropy loss is optimised. I did not draw the activation functions here to make the sketch clearer, but do keep in mind that typically after every convolutional or fully connected layer, an activation (e.g. ReLU) will be applied to all of the outputs.

Note the effect of a single $\mathrm{Conv}\rightarrow\mathrm{Pool}$ pass through the image: it reduces height and width of the individual channels in favour of their number, i.e. *depth*.

The softmax layer and cross-entropy loss are both introduced in more detail [in the previous tutorial](http://online.cambridgecoding.com/notebooks/cca_admin/deep-learning-for-complete-beginners-recognising-handwritten-digits). For summarisation purposes, a softmax layer's purpose is converting any vector of real numbers into a vector of *probabilities* (nonnegative real values that add up to 1). Within this context, the probabilities correspond to the likelihoods that an input image is a member of a particular class. Minimising the cross-entropy loss has the effect of maximising the model's confidence in the *correct* class, without being concerned for the probabilites for other classes---this makes it a more suitable choice for probabilistic tasks compared to, for example, the squared error loss.

## 4. Detour: Overfitting, regularisation and dropout

This will be the first (and hopefully the only) time when I will divert your attention to a seemingly unrelated topic. It regards a very important pitfall of machine learning---**overfitting** a model to the training data. While this is primarily going to be a major topic of the next tutorial in the series, the negative effects of overfitting will tend to become quite noticeable on the networks like the one we are about to build, and we need to introduce a way to properly protect ourselves against it, before going any further. Luckily, there is a very simple technique we can use.

Overfitting corresponds to adapting our model to the training set to such extremes that its generalisation potential (performance on samples outside of the training set) is *severely* limited. In other words, our model might have learned the training set (along with any noise present within it) perfectly, but it has failed to capture the underlying process that generated it. To illustrate, consider a problem of fitting a sine curve, with white additive noise applied to the data points:

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/plotsin.png)

Here we have a training set (denoted by blue circles) derived from the original sine wave, along with some noise. If we fit a degree-3 polynomial to this data, we get a fairly good approximation to the original curve. Someone might argue that a degree-14 polynomial would do better; indeed, given we have 15 points, such a fit would *perfectly* describe the training data. However, in this case, the additional parameters of the model cause catastrophic results: to cope with the inherent noise of the data, anywhere except in the closest vicinity of the training points, our fit is completely off.

Deep convolutional neural networks have a large number of parameters, especially in the fully connected layers. Overfitting might often manifest in the following form: if we don't have sufficiently many training examples, a small group of neurons might become responsible for doing most of the processing and other neurons becoming redundant; or in the other extreme, some neurons might actually become detrimental to performance, with several other neurons of their layer ending up doing nothing else but correcting for their errors.

To help our models generalise better in these circumstances, we introduce techniques of *regularisation*: rather than reducing the number of parameters, we impose *constraints* on the model parameters during training to keep them from learning the noise in the training data. The particular method I will introduce here is **dropout**---a technique that initially might seem like "dark magic", but actually helps to eliminate exactly the failure modes described above. Namely, dropout with parameter $p$ will, within a single training iteration, go through all neurons in a particular layer and, with probability $p$, *completely eliminate them from the network throughout the iteration*. This has the effect of forcing the neural network to cope with *failures*, and not to rely on existence of a particular neuron (or set of neurons)---relying more on a *consensus* of several neurons within a layer. This is a very simple technique that works quite well already for combatting overfitting on its own, without introducing further regularisers. An illustration is given below.

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/drop.png)

## Modern CNNs

How many think the CNNs would work on this?


![](https://lh3.googleusercontent.com/qZ6uF2t62wIbuovRoPwZ8_plopGdeYz7iGhzROPaI5Slk19PlT7HojPTCCJd9r0B2sJDernKJB0giA_l1xvo8Kh6rg=w640-h400-e365-rj-sc0x00ffffff)

To Do Something like this:


<img align="left" src="https://www.analyticsinsight.net/wp-content/uploads/2019/01/Object-Detection.jpg" width="500">


**We Need Power-UP**


<img src="https://i.imgflip.com/2c5xu5.jpg?a470976" width="800">

# 5. Progression in CNN Architecture

### Why Architectures Matter

CNN architectures are not one-size-fits-all solutions. The choice of architecture can significantly affect the outcome of your machine learning tasks. Here's why CNN architectures are crucial:

1. Task-specific Optimization: Different CNN architectures are tailored for specific tasks. Some are excellent at image classification, while others excel in object detection, image segmentation, or even natural language processing. Choosing the right architecture can save you valuable time and resources.

2. Model Efficiency: Architectures vary in terms of computational efficiency. Some are lightweight and suitable for deployment on resource-constrained devices, such as mobile phones or edge devices, while others are designed for high-performance cloud-based applications.

3. Generalization vs. Specialization: Architectures may generalize well to various tasks or specialize in particular domains. Understanding the strengths and weaknesses of each architecture allows you to choose the one best suited to your project.

4. Innovations and Research: CNN architectures have evolved over time due to continuous research and innovation. Staying updated with the latest architectures can give you a competitive edge in the field of AI/ML.

Today, we will explore the progression of CNN architectures, starting from historical milestones and moving into modern innovations. By the end of this section, you'll have a clear understanding of the role that architectures play in shaping the field of deep learning.

## Progression in CNN Architectures    
#### Objectives:
- Provide a deeper understanding of the innovations in each modern CNN architecture.
- Highlight key features and design choices that have led to improved performance.

Why?
- CNN architectures have evolved to address challenges like vanishing gradients, improved feature extraction, diverse features, etc.

1. ResNet(2015)

2. InceptionNet(2014, v2, v3):

3. Xception(2017)
4. MobileNet
5. EfficientNet
6. ResNeXt(2016)

Many More Concepts in Between...

In [ ]:
## Inception Module
import torch.nn as nn
import torch
import torch.nn.functional as F

1. ResNet(2015):
- Innovations:
    - Introduction of skip connections and residual learning.
    - Skip connections enable the training of much deeper networks by mitigating the vanishing gradient problem.
- Benefits:
    - ResNet architecture allows for the training of exceptionally deep networks (hundreds of layers) effectively.
    - State-of-the-art results in various image-related tasks.


<img align="left" src="https://upload.wikimedia.org/wikipedia/commons/b/ba/ResBlock.png" width="400" title='ResBlock'>

In [ ]:
## ResNet Block
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

2. InceptionNet(2014, v2, v3):
- Innovations:
    - Inception modules that perform convolution operations of different sizes simultaneously.
    - Captures features at multiple scales.
- Benefits:
    - Improved performance without significantly increasing the number of parameters.
        Inception architecture can adapt to different scales of features in the input data.

![InceptionBlock](https://miro.medium.com/v2/resize:fit:1400/1*KrBAIZjcrlXu6JPiPQj2vQ.jpeg)

In [ ]:
# InceptionA block for Inception v3
class InceptionA(nn.Module):
    def __init__(self, in_channels, pool_features):
        super(InceptionA, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size=1)

        self.branch5x5_1 = BasicConv2d(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = BasicConv2d(48, 64, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = BasicConv2d(96, 96, kernel_size=3, padding=1)

        self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)


3. Xception(2017):
- Innovations:
    - Depthwise separable convolutions, which separate spatial and channel-wise convolutions.
    - Xception (2017) is an extension of InceptionNet, focusing on depthwise separable convolutions.
    - Reduces the number of parameters and computational cost compared to standard convolutions.
- Benefits:
    - Achieves high accuracy with fewer parameters.
    - Suitable for applications with limited computational resources.

![DepthWiseConv](https://towardsdatascience.com/review-xception-with-depthwise-separable-convolution-better-than-inception-v3-image-dc967dd42568)
![DepthSepConv](https://www.paepper.com/blog/posts/depthwise-separable-convolutions-in-pytorch/depthwise-separable-convolution.png)

In [ ]:
## Xception Block
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, nin, nout, kernel_size, padding, bias=False):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=kernel_size, padding=padding, groups=nin, bias=bias)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

4. Squeeze-and-Excitation (SE) Blocks:

- Innovation: Adaptively recalibrate channel-wise feature responses.
- Explanation: SE blocks, as seen in MobileNetV3, enhance the representational power of a network by recalibrating features based on their importance, leading to improved performance.


![SqueezeExcite](https://miro.medium.com/v2/resize:fit:1120/1*bmObF5Tibc58iE9iOu327w.png)

In [ ]:
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduction_ratio):
        super(SqueezeExcitation, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction_ratio, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction_ratio, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

5. DenseNet(2017):

- Innovation: Connect each layer to every other layer.
- Explanation: DenseNet connects each layer to every other layer in a feed-forward fashion, promoting feature reuse and reducing the number of parameters.

![DenseNet](https://editor.analyticsvidhya.com/uploads/968542.png)

In [ ]:
class DenseBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_layers):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            self.layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
            in_channels += out_channels

    def forward(self, x):
        outputs = [x]
        for layer in self.layers:
            out = layer(torch.cat(outputs, 1))
            outputs.append(out)
        return torch.cat(outputs, 1)

6. ResNeXt(2016):
- Innovation:
    - Introduces a novel architecture design by using a "cardinality" parameter to control parallel paths within a block.
    - Aggregates residual blocks with different cardinalities, enhancing representational power and efficiency.
- Key Features:
    - Improves upon ResNet by replacing single convolutional layers with groups of convolutional layers.
    - Modular architecture allows adjusting cardinality for a balance between model size, performance, and computational cost.
    - Achieves state-of-the-art results in various computer vision tasks, showcasing its effectiveness.


![ResNeXt](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-06_at_4.32.52_PM.png)

![GroupConv](https://media.geeksforgeeks.org/wp-content/uploads/20210223134020/ps22-660x265.png)

In [ ]:
## ResNeXt Block
class ResNeXtBottleneck(nn.Module):
    """
    RexNeXt bottleneck type C (https://github.com/facebookresearch/ResNeXt/blob/master/models/resnext.lua)
    """
    def __init__(self, in_channels, out_channels, stride, cardinality, base_width, widen_factor):
        super(ResNeXtBottleneck, self).__init__()
        width_ratio = out_channels / (widen_factor * 64.)
        D = cardinality * int(base_width * width_ratio)
        self.conv_reduce = nn.Conv2d(in_channels, D, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn_reduce = nn.BatchNorm2d(D)
        self.conv_conv = nn.Conv2d(D, D, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn = nn.BatchNorm2d(D)
        self.conv_expand = nn.Conv2d(D, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn_expand = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut.add_module('shortcut_conv',
                                     nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, bias=False))
            self.shortcut.add_module('shortcut_bn', nn.BatchNorm2d(out_channels))

    def forward(self, x):
        bottleneck = self.conv_reduce.forward(x)
        bottleneck = F.relu(self.bn_reduce.forward(bottleneck), inplace=True)
        bottleneck = self.conv_conv.forward(bottleneck)
        bottleneck = F.relu(self.bn.forward(bottleneck), inplace=True)
        bottleneck = self.conv_expand.forward(bottleneck)
        bottleneck = self.bn_expand.forward(bottleneck)
        residual = self.shortcut.forward(x)
        return F.relu(residual + bottleneck, inplace=True)


7. MobileNet(2017, v2(2018), v3):
- Innovations:
    - Depthwise separable convolutions.
- Benefits:
    - Efficient for mobile and embedded devices.
    - Balances computational efficiency and accuracy.

![MobileNetv2](https://miro.medium.com/v2/resize:fit:1400/1*bqE59FvgpvoAQUMQ0WEoUA.png)

8. EfficientNet(2019):
- Innovations:
    - Compound scaling, optimizing depth, width, and resolution.
    - Module Similar to MobileNet
- Benefits:
    - Achieves state-of-the-art performance with fewer parameters.
    - Efficiently scales models to balance accuracy and computational cost.

![EfficientNet](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-06_at_10.45.54_PM.png)

In [ ]:
## EfficientNet Block
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, expand_ratio, kernel_size, stride, se_ratio):
        super(MBConv, self).__init__()
        self.stride = stride
        self.se_ratio = se_ratio
        self.expand = nn.Conv2d(in_channels, in_channels * expand_ratio, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels * expand_ratio)
        self.depthwise_conv = nn.Conv2d(in_channels * expand_ratio, in_channels * expand_ratio, kernel_size, stride=stride, padding=kernel_size//2, groups=in_channels * expand_ratio, bias=False)
        self.bn2 = nn.BatchNorm2d(in_channels * expand_ratio)
        self.se = SqueezeExcitation(in_channels * expand_ratio, int(in_channels * se_ratio))
        self.project = nn.Conv2d(in_channels * expand_ratio, out_channels, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.shortcut = (stride == 1 and in_channels == out_channels)

    def forward(self, x):
        identity = x
        x = F.relu(self.bn1(self.expand(x)), inplace=True)
        x = F.relu(self.bn2(self.depthwise_conv(x)), inplace=True)
        x = self.se(x)
        x = self.bn3(self.project(x))
        if self.shortcut:
            x += identity
        return x

# Inverted Residual Block for MobileNet v2
class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        hidden_dim = round(in_channels * expand_ratio)
        self.use_res_connect = stride == 1 and in_channels == out_channels

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(nn.Conv2d(in_channels, hidden_dim, 1, 1, 0, bias=False))
            layers.append(nn.BatchNorm2d(hidden_dim))
            layers.append(nn.ReLU6(inplace=True))

        layers.extend([
            # dw
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            # pw-linear
            nn.Conv2d(hidden_dim, out_channels, 1, 1, 0, bias=False),
            nn.BatchNorm2d(out_channels),
        ])

        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

In [ ]:
## Test
# Test input
x = torch.randn(1, 64, 32, 32)

# Test ResNet Block
resnet_block = ResBlock(64, 64)
out = resnet_block(x)
print(out.shape)  # Expected output: torch.Size([1, 64, 32, 32])

# Test InceptionA Block
inception_block = InceptionA(64, 32)
out = inception_block(x)
print(out.shape)  # Expected output: torch.Size([1, 256, 32, 32])

# Test InvertedResidual Block
inverted_residual_block = InvertedResidual(64, 64, 1, 6)
out = inverted_residual_block(x)
print(out.shape)  # Expected output: torch.Size([1, 64, 32, 32])

# Test DepthwiseSeparableConv Block
depthwise_separable_conv_block = DepthwiseSeparableConv(64, 128, 3, 1)
out = depthwise_separable_conv_block(x)
print(out.shape)  # Expected output: torch.Size([1, 128, 32, 32])

# Test ResNeXtBottleneck Block
resnext_bottleneck_block = ResNeXtBottleneck(64, 128, 1, 32, 4, 2)
out = resnext_bottleneck_block(x)
print(out.shape)  # Expected output: torch.Size([1, 128, 32, 32])

# Test MBConv Block
mbconv_block = MBConv(64, 128, 6, 3, 1, 0.25)
out = mbconv_block(x)
print(out.shape)  # Expected output: torch.Size([1, 128, 32, 32])

torch.Size([1, 64, 32, 32])
torch.Size([1, 256, 32, 32])
torch.Size([1, 64, 32, 32])
torch.Size([1, 128, 32, 32])
torch.Size([1, 128, 32, 32])
torch.Size([1, 128, 32, 32])


## 5. Let's implement a AutoEncoder!!

![](https://media.makeameme.org/created/the-power-is-5b5a12.jpg)

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        pass

        ###  YOUR CODE HERE  ###

    def forward(self, x):
      pass
        ###  YOUR CODE HERE  ###

## Coming UP... *U-Net*

Wonder What's in DALL-E/Stable Diffusion?

![](https://media.makeameme.org/created/i-wonder-what-18b1b8fc62.jpg)

# Play With Architectures in AE

### **HOMEWORK:**

Implement Variational Autoencoders (VAEs) and plot the intermediate images when moving from latent space of '5' to that of '1'.

In [ ]:
# Write code here